In [1]:
import os
import numpy as np
import progressbar
from imageio import imread

In [2]:
initial_path = '/Users/pedroprates/Google Drive/FaceRecognition/datasets/lfw/lfw_mtcnnpy_160'
os.listdir(initial_path)

dirs = [os.path.join(initial_path, d) for d in os.listdir(initial_path) if os.path.isdir(os.path.join(initial_path, d))]

In [3]:
import progressbar

inputs = []
for d in progressbar.progressbar(dirs):
    for f in os.listdir(d):
#         d = d.replace('/Users/pedroprates/Google Drive/', '/gdrive/My Drive/')
        if f.endswith('png') or f.endswith('jpg') or f.endswith('jpeg'):
            inputs.append(os.path.join(d, f))

100% (5749 of 5749) |####################| Elapsed Time: 0:00:00 Time:  0:00:00


In [16]:
outputs = []

for inp in progressbar.progressbar(inputs):
    filename = inp.split('/')[-1]
    path = inp.split('/')[:-1]
  
    filename = 'output_resnet/' + filename.split('.')[0] + '.npy'
    path = '/'.join(path)
#     path = path.replace('/Users/pedroprates/Google Drive/', '/gdrive/My Drive/')
  
    outputs.append(os.path.join(path, filename))

100% (13233 of 13233) |##################| Elapsed Time: 0:00:00 Time:  0:00:00


In [17]:
inputs_np = np.array(inputs)
outputs_np = np.array(outputs)

np.save('/Users/pedroprates/Google Drive/FaceRecognition/datasets/lfw/input_resnet_mac.npy', inputs_np)
np.save('/Users/pedroprates/Google Drive/FaceRecognition/datasets/lfw/output_resnet_mac.npy', outputs_np)

In [22]:
from imageio import imread

In [23]:
X = np.zeros((inputs_np.shape[0], 160, 160, 3))
y = np.zeros((outputs_np.shape[0], 512))

print("[RUNNING] X")
for ix, element in progressbar.progressbar(enumerate(inputs)):
    im = imread(element)
    X[ix, :, :, :] = im

print("[RUNNING] y Train")
for ix, element in progressbar.progressbar(enumerate(outputs)):
    em = np.load(element)
    y[ix, :] = em

| | #                                                | 52 Elapsed Time: 0:00:00

[RUNNING] X


| |          #                                    | 13232 Elapsed Time: 0:00:38
| | #                                               | 123 Elapsed Time: 0:00:00

[RUNNING] y Train


| |                             #                 | 13232 Elapsed Time: 0:00:12


In [27]:
np.save('/Users/pedroprates/Google Drive/FaceRecognition/datasets/lfw/X.npy', X)
np.save('/Users/pedroprates/Google Drive/FaceRecognition/datasets/lfw/y.npy', y)

# Teste

In [5]:
import keras
import os
import tensorflow as tf
import progressbar
import numpy as np
from imageio import imread

In [6]:
import keras.backend as K
# Custom loss function
def distillation_loss(y_true, y_pred):
    return K.square(y_pred - y_true)

def max_diff(y_true, y_pred):
    return K.max(K.square(y_pred - y_true), axis=-1)

def sum_diff(y_true, y_pred):
    return K.sum(K.square(y_pred - y_true), axis=-1)

keras.losses.distillation_loss = distillation_loss
keras.metrics.max_diff = max_diff
keras.metrics.sum_diff = sum_diff

In [7]:
model = keras.models.load_model('/Users/pedroprates/Google Drive/FaceRecognition/models/mobile-net/mobilenetv1_v2.h5')

In [8]:
base_path = '/Users/pedroprates/Google Drive/FaceRecognition/datasets/lfw/lfw_mtcnnpy_160/'
list_folders = os.listdir(base_path)
list_folders = [os.path.join(base_path, x) for x in list_folders]

for folder in progressbar.progressbar(list_folders):
    if not os.path.isdir(folder):
        continue
    
    list_images = os.listdir(folder)
    list_images = [os.path.join(folder, image) for image in list_images]
    list_images = list(filter(lambda x: os.path.isfile(x), list_images))
    list_images = list(filter(lambda x: '.DS_Store' not in x, list_images))
    filenames = [x.split('/')[-1].split('.')[0] for x in list_images]
    output_filenames = [x + '.npy' for x in filenames]
    output_folder = os.path.join(folder, 'mobilenetv1_v2')

    # Get the embeddings
    images = np.array([imread(f) / 255 for f in list_images])
    embeddings = model.predict(images)
  
    if not os.path.exists(os.path.join(base_path, output_folder)):
        os.makedirs(os.path.join(base_path, output_folder))
    for idx, embedding in enumerate(embeddings):
        emb_to_save = embedding.reshape(1, *embedding.shape)
        np.save(os.path.join(output_folder, output_filenames[idx]), emb_to_save)

100% (5755 of 5755) |####################| Elapsed Time: 0:24:28 Time:  0:24:28


In [9]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 160, 160, 3)       0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, 161, 161, 3)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 80, 80, 32)        864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 80, 80, 32)        128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 80, 80, 32)        0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 80, 80, 32)        288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 80, 80, 32)        128       
__________

In [10]:
model16 = keras.models.load_model('/Users/pedroprates/Google Drive/FaceRecognition/models/mobile-net/mobilenetv1_v16.h5')

In [13]:
import json

model_json = model.to_json()

with open('models/mobile-net/json-test/modelv2.json', 'w') as f:
    json.dump(model_json, f)

In [14]:
model.save_weights('models/mobile-net/json-test/modelv2_weights.h5')